Download and install packages

In [ ]:
!pip install spacy
!python -m spacy download nl_core_news_sm

Importing libraries

In [ ]:
# Importing libraries
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy  as np
import string
import re
import pickle

#NLP
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

import spacy
from spacy.symbols import nsubj, VERB
from spacy.lang.nl.stop_words import STOP_WORDS

nlp = spacy.load("nl_core_news_sm")
lemmatizer = nltk.stem.WordNetLemmatizer()

Cleaning

In [ ]:
# additional stopwords obtained from https://github.com/stopwords-iso/stopwords-nl/blob/master/stopwords-nl.txt
nl_stopwords = pd.read_csv('stopwords-nl.txt')
# flatten list and strip spaces
nl_stopwords = [item.strip() for sublist in nl_stopwords.values.tolist() for item in sublist]

In [ ]:
# loading dataset
df_complete = pd.read_pickle("data_randomized_complete_correct.pkl")

In [ ]:
# apply to full_text column
df_complete = df_complete.dropna(subset=['full_text'])

# Remove rows where the length of text in 'text' column is less than 5 characters
df_complete = df_complete[df_complete['full_text'].str.len() >= 5]

# Remove rows where the text is exactly "Nee, ik heb geen opmerkingen"
df_complete = df_complete[df_complete['full_text'] != "Nee, ik heb geen opmerkingen"]

In [ ]:
# Load the Dutch NLP model
nlp = spacy.load("nl_core_news_sm")

# Update Spacy's default stop words with additional custom Dutch stopwords
for word in nl_stopwords:
    nlp.Defaults.stop_words.add(word)

english_stopwords = set(nltk.corpus.stopwords.words('english'))
nlp.Defaults.stop_words.update(english_stopwords)

def clean_and_lemmatize(text):
    # Lowercase the text to normalize it
    text = text.lower()

    # Remove new line characters and other unnecessary whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove all numbers (digits)
    text = re.sub(r'\d+', '', text)

    # Process the cleaned text through Spacy, filtering out stop words and punctuation
    doc = nlp(text)
    cleaned_text = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])

    return cleaned_text

In [ ]:
df_complete['clean_text'] = df_complete['full_text'].apply(clean_and_lemmatize)

In [ ]:
# saving as pickle
df_complete.to_pickle('data_randomized_complete_cleaned.pkl')